# Infer ***cellmask*** -  1️⃣ 

> WARNING: (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)


>> WARNING:  THIS DOES NOT WORK WELL - lacking fluorescent marker
>> #### Because we do NOT have a direct cell membrane / cellmask signal, this segmentation is trickiest and potentially most problematic part of the overall sub-cellular component inference pipeline. We are using the nuclei of the cell with the brightest total fluorescence (all channels) to identify a single cellmask for all downstream steps. The Cellmask (via the Cytosol mask) will be used to define ALL subsequent sub-cellular Objects.

--------------

## OBJECTIVE: 
### ✅ Infer sub-cellular component #2: ***cellmask***/cell body in order to understand interactome 

Infer a segmentation of the cell body -- the ***cellmask*** -- in order to measure its shape, position, and size.

CONTEXT: "Cellmask" is used here becuase subsequent experiments will contain neurons who's cellmask has a similar shape to an iPS cell body.

## OVERVIEW: 

This method is used in the case where there is no cell fill/membrane marker.

We will infer the cellmask from a combination of fluorescent signals. The current selection includes the lysosomes, ER, and Golgi (e.g., 'Ch = 1, 3, 5) which have some intracellular fluorescence - likely from off target marker localization to the entire cellmask and/or the cell membrane. There are two other channels, the residual channel from linear unmixing (e.g. `ch = 7`) and the lipid droplet channel (e.g., `ch = 6`) that could more lead to more unbiased selection of the entire cellmask. However, the drawback of these two markers is that they are present in every cell which makes downstream cell selection more challenging. To expand on this, we will be collecting per cell measurements, so each cell area has to be segmented individually even if there are two appropriately labeled cells within one field of view. 

## preamble

1. imports
2. setup
4. infer-cellmask
    * input
    * pre-processing
    * core processing
    * post-processing
    * select individual cell
    * output



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np

from scipy import ndimage as ndi
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling

from skimage import filters
from skimage.segmentation import watershed
from skimage.morphology import remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))


from infer_subc.core.file_io import (read_czi_image,
                                                                    list_image_files)

from infer_subc.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROX_CH ,
                                                                    ER_CH ,
                                                                    LD_CH ,
                                                                    RESIDUAL_CH )                                                                    
from infer_subc.core.img import *

from infer_subc.organelles import fixed_get_optimal_Z_image, fixed_find_optimal_Z, find_optimal_Z

%load_ext autoreload
%autoreload 2

## SETUP
CUSTOMIZE WITH: 
1. updated path to data
2. updated folder name for "raw" data

> NOTE: we are operating on a single "test" image in this notebook.  The batch-processing of all the images will be happen at the end of the notebook after we have developed/confirmed the setmentation procedures and parameter settings.

In [2]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
# CUSTOMIZE HERE --->
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/infer-subc"

# linearly unmixed ".czi" files are here
# CUSTOMIZE HERE --->
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [3]:
# isolate image as an ndarray and metadata as a dictionary
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

c:\Users\Shannon\Anaconda3\envs\infer-subc\lib\site-packages\ome_types\_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


##  infer ***cellmask***

#### summary of steps

➡️ INPUT
- multi-channel sum (6.*1, 3, 2.*5)
- nuclei mask

PRE-PROCESSING
- rescaling
- denoise/smoothing
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr edge filtered intensity

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

OUTPUT ➡️ 
- mask of CELLMASK


> #### Note: this pipeline will eventually include a selection step to identify the cellmask that are properly labeled with all fluorescent markers. This could be one single cell per image, or more if applicable data is available.

## INPUT (prototype)

Combine multiple channels that will allow inference of the cellmask.

Note: the selected channels were chosen based on their qualitative ability to fill the cytoplasmic area of the cell. 

In [4]:
###################
# INPUT
###################
struct_img_raw = (6. * img_data[LYSO_CH].copy() +
                  1. * img_data[ER_CH].copy() + 
                  2. * img_data[GOLGI_CH].copy())

raw_nuclei = img_data[NUC_CH].copy() 

print(struct_img_raw.shape)
print(raw_nuclei.shape)

(17, 704, 704)
(17, 704, 704)


### VISUALIZE: the composite image that will be used as input
Use this to adjust which channels and multiplication factors to use to create the composite input image above.

In [5]:
viewer = napari.Viewer()
viewer.add_image(
    img_data,
    scale=scale
)

<Image layer 'img_data' at 0x25e5031c3a0>

In [6]:
viewer.add_image(
    struct_img_raw,
    scale=scale
)

<Image layer 'struct_img_raw' at 0x25e527b0430>

<code style="background:yellow;color:black">Consider moving this to the definition section below</code>

In [7]:
def _raw_cellmask_MCZ(img_in):
    """ define cellmask image
    """
    CELLMASK_W = (6.,1.,2.)
    CELLMASK_CH = (LYSO_CH,ER_CH,GOLGI_CH)
    img_out = np.zeros_like(img_in[0]).astype(np.double)
    for w,ch in zip(CELLMASK_W,CELLMASK_CH):
        img_out += w*img_in[ch]
    return img_out

_struct_img_raw = _raw_cellmask_MCZ(img_data)

viewer.add_image(
    _struct_img_raw,
    scale=scale
)


<Image layer '_struct_img_raw' at 0x25e50a57340>

## PRE-PROCESSING (prototype)


In [10]:

###################
# PRE_PROCESSING
###################

################# smoothing
cellmask_norm = min_max_intensity_normalization(struct_img_raw)

med_filter_size = 15
cellmask_med = median_filter_slice_by_slice(cellmask_norm, 
                                        size=med_filter_size)

## Need to adjust image_smoothing_gaussian_slice_by_slice in pre_processing_utils.py to integrate the mode

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0
cellmask_guas = ndi.gaussian_filter(cellmask_med,
                                sigma=gaussian_smoothing_sigma,
                                mode="nearest", 
                                truncate=gaussian_smoothing_truncate_range)


################# NON-Linear aggregation
cellmask_log, d = log_transform(cellmask_guas) 
cellmask_log_norm = intensity_normalization(cellmask_log, scaling_param=[0])

# cellmask_edges = filters.difference_of_gaussians(cellmask_log_norm, 2)

# composite_cellmask = intensity_normalization(cellmask_edges, scaling_param=[0]) + cellmask_log_norm 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [9]:
viewer.add_image(
    cellmask_med,
    scale=scale
)
viewer.add_image(
    cellmask_guas,
    scale=scale
)
viewer.add_image(
    cellmask_log_norm,
    scale=scale
)
# viewer.add_image(
#     cellmask_edges,
#     scale=scale
# )
# viewer.add_image(
#     composite_cellmask,
#     scale=scale
# )

<Image layer 'composite_cellmask' at 0x25e437c8bb0>

<code style="background:yellow;color:black">Consider moving this to the definition section below</code>

In [11]:
def _non_linear_cellmask_transform(in_img):
    """ non-linear distortion to fill out cellmask
    log + edge of smoothed composite
    """
    # non-Linear processing
    log_img, d = log_transform(in_img.copy()) 
    return intensity_normalization(log_img,scaling_param=[0])
    # return intensity_normalization(filters.difference_of_gaussians(log_img, 2),scaling_param=[0])  + log_img

_log_cellmask = _non_linear_cellmask_transform(cellmask_guas)

viewer.add_image(
    _log_cellmask,
    scale=scale
)


intensity normalization: min-max normalization with NO absoluteintensity upper bound


<Image layer '_log_cellmask' at 0x25e437e2d90>

<code style="background:yellow;color:black">Eventually this will be replaced by calling the un-name mangled version of the infer nuclei code that will be in infer-subc.organelles.</code>

In [12]:
################# part 2 Nuclei pre-process
def _infer_nuclei_3D( in_img: np.ndarray,
                       median_sz: int, 
                       gauss_sig: float,
                       thresh_factor: float,
                       thresh_min: float,
                       thresh_max: float,
                       max_hole_w: int,
                       small_obj_w: int,
                       sz_filter_method: str
                     ) -> np.ndarray:
    """
    Procedure to infer 3D nuclei segmentation from multichannel z-stack input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    cellmask_mask: Optional[np.ndarray] = None
        mask
    median_sz: int
        width of median filter for signal
    gauss_sig: float
        sigma for gaussian smoothing of  signal
    thresh_factor: float
        adjustment factor for log Li threholding
    thresh_min: float
        abs min threhold for log Li threholding
    thresh_max: float
        abs max threhold for log Li threholding
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    sz_filter_method: str
        method for size filtering; either "3D" or "slice_by_slice"

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    nuclei = select_channel_from_raw(in_img, nuc_ch)


    ###################
    # PRE_PROCESSING
    ###################                
    nuclei = min_max_intensity_normalization(nuclei)
    nuclei = median_filter_slice_by_slice(nuclei,
                                          size=median_sz)
    nuclei = image_smoothing_gaussian_slice_by_slice(nuclei,
                                                     sigma=gauss_sig )


    ###################
    # CORE_PROCESSING
    ###################
    nuclei_object = apply_log_li_threshold(nuclei, 
                                           thresh_factor=thresh_factor, 
                                           thresh_min=thresh_min, 
                                           thresh_max=thresh_max)


    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = hole_filling(nuclei_object, 
                                 hole_min=0, 
                                 hole_max=max_hole_w**2, 
                                 fill_2d=True)

    nuclei_object = size_filter(nuclei_object, 
                                min_size = small_obj_w**3, 
                                method = sz_filter_method,
                                connectivity=1)


    return nuclei_object


def _fixed_infer_nuclei_3D(in_img: np.ndarray) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    cellmask_mask: np.ndarray
        mask
 
    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    nuc_ch = NUC_CH
    median_sz = 4     
    gauss_sig = 1.34
    threshold_factor = 0.9
    thresh_min = 0.1
    thresh_max = 1.0
    max_hole_w = 5
    small_obj_w = 15
    sz_filter_method = "3D"

    return _infer_nuclei_3D( in_img,
                             median_sz,
                             gauss_sig,
                             threshold_factor,
                             thresh_min,
                             thresh_max,
                             max_hole_w,
                             small_obj_w,
                             sz_filter_method )


_NU_object =  _fixed_infer_nuclei_3D(img_data) 
NU_labels = label(_NU_object)

<code style="background:yellow;color:black">Consider moving this to the definition section below</code>

In [13]:

def _masked_object_thresh(
    structure_img_smooth: np.ndarray, th_method: str, cutoff_size: int, th_adjust: float
) -> np.ndarray:
    """
    wrapper for applying Masked Object Thresholding with just two parameters via `MO` from `aicssegmentation`
    Parameters
    ------------
    structure_img_smooth: np.ndarray
        a 3d image
    th_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    cutoff_size: 
        Masked Object threshold `size_min`
    th_adjust: 
        Masked Object threshold `local_adjust`

    Returns
    -------------
        np.ndimage 

    """

    struct_obj = MO(
        structure_img_smooth,
        global_thresh_method=th_method,
        object_minArea=cutoff_size,
        extra_criteria=True,
        local_adjust=th_adjust,
        return_object=False,
        dilate=True,
    )
    return struct_obj


## CORE PROCESSING

In [14]:
###################
# CORE_PROCESSING
###################
low_level_min_size =  50

# ################# part 1
cellmask_binary = _masked_object_thresh(cellmask_log_norm, 
                           th_method='ave', 
                           cutoff_size=low_level_min_size, 
                           th_adjust= 0.15)                                           

In [15]:
viewer.add_image(
    cellmask_binary,
    scale=scale
)

<Image layer 'cellmask_binary' at 0x25e438999d0>

## POST-PROCESSING

<code style="background:yellow;color:black">Watershed (with or without the nuclei as a marker) does not work well here to get an instance segmentation</code>

In [16]:
###################
# POST_PROCESSING
###################
hole_width = 100
removed_holes = hole_filling(cellmask_binary, 
                            hole_min=0, 
                            hole_max=hole_width**2,
                            fill_2d = True) 


small_object_width = 45
cleaned_img = size_filter_2D(removed_holes, 
                             min_size= small_object_width**3, 
                             connectivity=1)


watershed_mask = cleaned_img 
inverted_img = 1. - cleaned_img

labels_out = watershed(
            inverted_img,
            markers=NU_labels,
            connectivity= 3, # np.ones((1,3,3), bool),
            mask=watershed_mask,
            )

In [17]:
viewer.add_image(
    removed_holes,
    scale=scale
)
viewer.add_image(
    cleaned_img,
    scale=scale
)
viewer.add_image(
    inverted_img,
    scale=scale
)
viewer.add_labels(
    labels_out,
    scale=scale
)

<Labels layer 'labels_out' at 0x25e52d71a60>

<code style="background:yellow;color:black">Consider moving this to the definition section below</code>

In [19]:
def _masked_inverted_watershed(img_in, markers, mask):
    """wrapper for watershed on inverted image and masked

    """
    labels_out = watershed(
                1. - img_in,
                markers=markers,
                connectivity=3,
                mask=mask,
                )
    return labels_out


_labels_out = _masked_inverted_watershed(cleaned_img, NU_labels, cleaned_img)

viewer.add_labels(
    _labels_out,
    scale=scale
)

<Labels layer '_labels_out [1]' at 0x25e43b8f550>

## POST POST-PROCESSING

<code style="background:yellow;color:black">This should eventually be altered to maintain all cells that have the correct number of organelle markers</code>

What is we used the nuclei (or an expanded nuclei) to select the cells with the most signal, then after selecting the nuclei with signal, use those as the watershed marker - I think it may resolve some conflicts, but not all

In [20]:
###################
# POST- POST_PROCESSING
###################
# keep the "CELLMASK" label which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [cellmask_norm[labels_out == label].sum() for label in all_labels]
keep_label = all_labels[np.argmax(total_signal)]
keep_label

cellmask_out = np.zeros_like(labels_out)
cellmask_out[labels_out==keep_label] = 1

viewer.add_image(
    cellmask_out,
    scale=scale
)

<Image layer 'cellmask_out' at 0x25e52975190>

In [21]:
def _choose_max_label(raw_signal: np.ndarray, labels_in: np.ndarray):
    """ keep only the label with the maximum raw signal

    """

    all_labels = np.unique(labels_in)[1:]

    total_signal = [raw_signal[labels_in == label].sum() for label in all_labels]
    # combine NU and "labels" to make a CELLMASK
    keep_label = all_labels[np.argmax(total_signal)]

    labels_max = np.zeros_like(labels_in)
    labels_max[labels_in==keep_label] = 1
    return labels_max

_cellmask_out = _choose_max_label(cellmask_norm,labels_out)

viewer.add_image(
    _cellmask_out,
    scale=scale
)

<Image layer '_cellmask_out' at 0x25e43926160>



<code style="background:yellow;color:black">***WIP*** 2D-->3D transition stops here</code>

## DEFINE parameterized  `_infer_cellmask` function

A function to infer_cellmask from our (Channel, 1 Z slice, X, Y) image accourding the the following parameters: 
-  

In [37]:
##########################
# 1. infer_cellmask
##########################

def _infer_cellmask_3D(in_img: np.ndarray,
    nuclei_labels: np.ndarray,
    median_sz_cellmask: int,
    gauss_sig_cellmask: float,
    gauss_truc_rang: float,
    mo_method: str,
    mo_adjust: float,
    mo_cutoff_size: int,
    max_hole_w_cellmask: int,
    small_obj_w_cellmask: int
) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuclei_labels:
        a 3d mask of nuclei
    median_sz_cellmask: 
        width of median filter for _cellmask_ signal
    gauss_sig_cellmask: 
        sigma for gaussian smoothing of _cellmask_ signal
    gauss_truc_rang:
        cutoff value for gaussian
    mo_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    mo_adjust: 
        Masked Object threshold `local_adjust`
    mo_cutoff_size: 
        Masked Object threshold `size_min`
    max_hole_w_cellmask: 
        hole filling cutoff for cellmask signal post-processing
    small_obj_w_cellmask: 
        minimu object size cutoff for cellmask signal post-processing

    Returns
    -------------
    cellmask_mask:
        a logical/labels object defining boundaries of cellmask

    """

    ###################
    # EXTRACT
    ###################
    struct_img = _raw_cellmask_MCZ(in_img)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=1
    )


    ###################
    # PRE_PROCESSING
    ###################                         
    ################# part 1- cellmask
    struct_img = min_max_intensity_normalization(struct_img)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=2
    )

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    struct_img = median_filter_slice_by_slice(struct_img, 
                                              size=median_sz_cellmask)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=3
    )

    struct_img = ndi.gaussian_filter(struct_img,
                                     sigma=gauss_sig_cellmask,
                                     mode="nearest",
                                     truncate=gauss_truc_rang)
    viewer.add_image(
    struct_img,
    scale=scale,
    name=4
    )


    # struct_img_non_lin, d = log_transform(struct_img)
    # struct_img_non_lin = intensity_normalization(struct_img_non_lin, scaling_param=[0])
    struct_img_non_lin = _non_linear_cellmask_transform(struct_img)
    viewer.add_image(
    struct_img_non_lin,
    scale=scale,
    name=5
    )
   
    ###################
    # CORE_PROCESSING
    ###################    
    struct_obj = _masked_object_thresh(struct_img_non_lin, 
                                       th_method=mo_method, 
                                       cutoff_size=mo_cutoff_size, 
                                       th_adjust=mo_adjust)               
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=6
    )

    ###################
    # POST_PROCESSING
    ###################
    struct_obj = hole_filling(struct_obj, 
                              hole_min =0 , 
                              hole_max=max_hole_w_cellmask**2, 
                              fill_2d = True) 
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=7
    )

    struct_obj = size_filter_2D(struct_obj, 
                                min_size= small_obj_w_cellmask**3, 
                                connectivity=1)
    viewer.add_image(
    struct_obj,
    scale=scale,
    name=8
    )

    labels_out = _masked_inverted_watershed(struct_obj, nuclei_labels, struct_obj)
    viewer.add_image(
    labels_out,
    scale=scale,
    name=9
    )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "CELLMASK" label which contains the highest total signal
    cellmask_out = _choose_max_label(struct_img, labels_out)

    return cellmask_out




## DEFINE `_fixed_infer_cellmask` function

Based on the _prototyping_ above define the function to infer cellmask. with a *fixed* set of parameters for each step in the procedure.  That is they are all "hard coded"

In [34]:
##########################
# 1. fixed_infer_cellmask
##########################


def _fixed_infer_cellmask_3D(in_img: np.ndarray, nuclei_labels: np.ndarray) -> np.ndarray:
    """
    Procedure to infer cellmask from linearly unmixed input, with a *fixed* set of parameters for each step in the procedure.  i.e. "hard coded"

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    nuclei_object:
        a 3d mask of nuclei

    Returns
    -------------
    cellmask_mask:
        a logical/labels object defining boundaries of cellmask
    """
    

    ###################
    # PARAMETERS
    ###################   
    median_sz_cellmask = 15
    gauss_sig_cellmask = 1.34
    gauss_truc_rang = 3.0
    mo_method = "ave"
    mo_adjust = 0.15
    mo_cutoff_size = 50
    max_hole_w_cellmask = 100
    small_obj_w_cellmask = 45

    cellmask_out = _infer_cellmask_3D(in_img,
                              nuclei_labels,
                              median_sz_cellmask,
                              gauss_sig_cellmask,
                              gauss_truc_rang,
                              mo_method,
                              mo_adjust,
                              mo_cutoff_size,
                              max_hole_w_cellmask,
                              small_obj_w_cellmask) 

    return cellmask_out




In [38]:

SO_label =  _fixed_infer_cellmask_3D(img_data, NU_labels) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [39]:
viewer.add_image(
    SO_label,
    scale=scale
)

<Image layer 'SO_label [1]' at 0x25ea42fd970>

---------------------
# TEST `_infer_cellmask`  function defined above

<code style="background:yellow;color:black">***WIP*** 2D-->3D transition stops here</code>
##


In [ ]:
from infer_subc.organelles import fixed_infer_cellmask, infer_cellmask

cellmask_ =  fixed_infer_cellmask(img_2D) 

In [ ]:
viewer.add_image(
    SO_label,
    scale=scale
)
viewer.add_image(
    cellmask_,
    scale=scale 
)

Write the `infer_cellmask` spec to the widget json

In [ ]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_cellmask =  {
        "name": " infer cellmask mask (fixed parameters)",
        "python::module": "infer_subc.organelles",
        "python::function": "fixed_infer_cellmask",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_cellmask", _fixed_infer_cellmask, overwrite=True)

In [ ]:

_infer_cellmask =  {
        "name": " infer cellmask mask",
        "python::module": "infer_subc.organelles",
        "python::function": "infer_cellmask",
        "parameters": {
                "median_sz_cellmask": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig_cellmask": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "median_sz_nuc": {
                        "widget_type": "slider",
                        "data_type": "int",
                        "min": 3,
                        "max": 15,
                        "increment": 1
                },
                "gauss_sig_nuc": {
                        "data_type": "float",
                        "increment": 0.25,
                        "max": 15.0,
                        "min": 1.25,
                        "widget_type": "slider"
                },
                "mo_method": {
                        "data_type": "str",
                        "widget_type": "drop-down",
                        "options": [
                                "triangle",
                                "median",
                                "ave_tri_med"
                                ]
                },
                "mo_adjust": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "mo_cutoff_size": {
                        "data_type": "int",
                        "increment": 10,
                        "max": 250,
                        "min": 10,
                        "widget_type": "slider"
                },
                "thresh_factor": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.2,
                        "min": 0.6,
                        "widget_type": "slider"
                },
                "thresh_min": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": .9,
                        "min": 0.0,
                        "widget_type": "slider"
                },
                "thresh_max": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 1.0,
                        "min": 0.1,
                        "widget_type": "slider"
                },
                "max_hole_w_nuc": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 40,
                        "min": 4,
                        "widget_type": "slider"
                },           
                "small_obj_w_nuc": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                },                           
                "max_hole_w_cellmask": {
                        "data_type": "int",
                        "increment": 2,
                        "max": 100,
                        "min": 20,
                        "widget_type": "slider"
                },           
                "small_obj_w_cellmask": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                },        
        }
}

add_function_spec_to_widget_json("infer_cellmask", _infer_cellmask, overwrite=True )



In [ ]:

_raw_cellmask_MCZ =  {
        "name": "define weighted aggregate cellmask signal (MCZ-cellprofiler)",
        "python::module": "infer_subc.organelles",
        "python::function": "raw_cellmask_MCZ",
        "parameters": None
        }

add_function_spec_to_widget_json("raw_cellmask_MCZ", _raw_cellmask_MCZ, overwrite=True)

In [ ]:

_non_linear_cellmask_transform =  {
        "name": "non-linear filter of cellmask signal (MCZ-cellprofiler)",
        "python::module": "infer_subc.organelles",
        "python::function": "non_linear_cellmask_transform",
        "parameters": None
        }

add_function_spec_to_widget_json("non_linear_cellmask_transform", _non_linear_cellmask_transform)

In [ ]:

_masked_inverted_watershed =  {
        "name": "watershed on inverted image and masked",
        "python::module": "infer_subc.organelles",
        "python::function": "masked_inverted_watershed",
        "parameters": None
        }

add_function_spec_to_widget_json("masked_inverted_watershed", _masked_inverted_watershed)

In [ ]:

_choose_max_label =  {
        "name": "keep only the label with the maximum raw signa",
        "python::module": "infer_subc.core.img",
        "python::function": "choose_max_label",
        "parameters": None
        }

add_function_spec_to_widget_json("choose_max_label", _choose_max_label)

In [ ]:

_min_max_intensity_normalization =  {
        "name": "Min Max Intesity Normalization",
        "python::module": "infer_subc.core.img",
        "python::function": "min_max_intensity_normalization",
        "parameters": None
        }

add_function_spec_to_widget_json("min_max_intensity_normalization", _min_max_intensity_normalization)

In [ ]:
from infer_subc.organelles_config.helper import add_function_spec_to_widget_json

_masked_object_thresh =  {
        "name": "Masked Object Threshold wrapper for widgets",
        "python::module": "infer_subc.core.img",
        "python::function": "masked_object_thresh",
        "parameters": {
                "th_method": {
                        "data_type": "str",
                        "widget_type": "drop-down",
                        "options": [
                        "triangle",
                        "median",
                        "ave_tri_med"
                        ]
                },
                "cutoff_size": {
                        "data_type": "int",
                        "widget_type": "slider",
                        "min": 0,
                        "max": 2000,
                        "increment": 50
                },
                "th_adjust": {
                        "data_type": "float",
                        "widget_type": "slider",
                        "min": 0,
                        "max": 2,
                        "increment": 0.02
                }
        }
}

add_function_spec_to_widget_json("masked_object_thresh", _masked_object_thresh, overwrite=True)

--------------------------

# TEST `infer_cellmask` exported functions


In [ ]:
from infer_subc.organelles import fixed_infer_cellmask

cellmask_mask =  fixed_infer_cellmask(img_2D) 

## Visualize  2


In [ ]:
# viewer = napari.Viewer()

viewer.scale_bar.visible = True

viewer.add_labels(
    cellmask_mask,
    scale=scale,
    blending='additive'
)


In [ ]:

from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)
viewer.close()

-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
def make_infer_cellmask_step_by_step_dict():
    """
    crete .json version of infer_cellmask
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []

    ###################
    # EXTRACT
    ###################   
    # struct_img = _raw_cellmask_MCZ(in_img)
    step_name.append("1")
    function_name.append("raw_cellmask_MCZ")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(0)

    ###################
    # PRE_PROCESSING
    ###################
    #CELLMASK
    step_name.append("3")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(1)


    step_name.append("4")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 15 ))
    parent.append(3)


    step_name.append("5")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(4)

    step_name.append("6")
    function_name.append("non_linear_cellmask_transform")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    #NUC
    step_name.append("7")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(2)

    step_name.append("8")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(7)

    step_name.append("9")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(8)

    ###################
    # CORE_PROCESSING
    ###################
    # CELLMASK
    step_name.append("10")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(6)


    # NUCLEI
    step_name.append("11")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(9)


    ###################
    # POST_PROCESSING
    ###################
    # NUCLEI
    step_name.append("12")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(11)

    step_name.append("13")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(12)

    step_name.append("14")
    function_name.append("label")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(13)

    # CELLMASK
    step_name.append("15")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=25**2, fill_2d=True))
    parent.append(10)

    step_name.append("16")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(15)

    step_name.append("17")
    function_name.append("masked_inverted_watershed")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([ 5 , 14, 16])

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "CELLMASK" label which contains the highest total signal
    step_name.append("18")
    function_name.append("choose_max_label")
    category.append("postpostprocessing")
    parameter_values.append(None)
    parent.append([5, 17])

    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc.organelles_config.helper import write_workflow_json

infer_cellmask_stepbystep_dict = make_infer_cellmask_step_by_step_dict()

write_workflow_json("conf_2.1.cellmask_stepbystep", infer_cellmask_stepbystep_dict )

In [ ]:
def make_infer_cellmask_step_by_step_from_raw_dict():
    """
    Procedure to infer nuclei from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    cellmask_mask: np.ndarray
        mask

    Returns
    -------------
    nuclei_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []

    ###################
    # EXTRACT
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    # CELLMASK
    step_name.append("2")
    function_name.append("raw_cellmask_MCZ")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(1)

    # NUC
    step_name.append("3")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(1)

    ###################
    # PRE_PROCESSING
    ###################
    # CELLMASK
    step_name.append("4")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(2)

    step_name.append("5")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 15 ))
    parent.append(4)

    step_name.append("6")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(5)

    step_name.append("7")
    function_name.append("non_linear_cellmask_transform")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(6)

    #NUC
    step_name.append("8")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(3)

    step_name.append("9")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(8)

    step_name.append("10")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.4 ))
    parent.append(9)

    ###################
    # CORE_PROCESSING
    ###################
    # CELLMASK
    step_name.append("11")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(7)

    # NUC
    step_name.append("12")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(10)

    ###################
    # POST_PROCESSING
    ###################
    # nUC
    step_name.append("13")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=5**2, fill_2d=True))
    parent.append(12)

    step_name.append("14")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(13)

    step_name.append("15")
    function_name.append("label")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append(14)


    # CELLMASK
    step_name.append("16")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=25**2, fill_2d=True))
    parent.append(11)

    step_name.append("17")
    function_name.append("size_filter_2D")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 15**2  ))
    parent.append(16)

    step_name.append("18")
    function_name.append("masked_inverted_watershed")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([  7 , 15,17 ])

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "CELLMASK" label which contains the highest total signal
    step_name.append("19")
    function_name.append("choose_max_label")
    category.append("postpostprocessing")
    parameter_values.append(None)
    parent.append([6,18])

    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_cellmask_stepbystep_from_raw_dict = make_infer_cellmask_step_by_step_from_raw_dict()

write_workflow_json("conf_1.1.cellmask_stepbystep_from_raw", infer_cellmask_stepbystep_from_raw_dict)

-------------
## SUMMARY

The above explains the overall framework.  

### NEXT: INFER NUCLEI

proceed to [02_infer_nuclei.ipynb](./02_infer_nuclei.ipynb)
